In [ ]:
YDSHPERE = False

In [ ]:
if YDSHPERE:
    %pip install efficientnet_pytorch
else:
    !pwd

In [ ]:
#!g1.1
import os
import gc
import sys
import cv2
import glob
import time
import json
import random
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn import metrics
from tqdm.notebook import tqdm
from efficientnet_pytorch import model as enet
import albumentations as A
#warnings.filterwarnings('ignore', category=UserWarning) 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')

In [ ]:
VER = 'v6'
DEBUG = False
PARAMS = {
    'version': VER,
    'img_size': 380,
    'mixup': True,
    'folds': 4,
    'folds_train': None,
    'batch_size': 16,
    'workers': 16,
    'epochs': 4 if DEBUG else 40,
    'patience': 4,
    'warmup': False,
    'backbone': 'efficientnet-b1', # 'efficientnet-bX' or 'resnext'
    'dropout': .4,
    'seed': 2020,
    'lr': .0005,
    'comments': ''
}
DATA_PATH = './data'
IMGS_PATH = f'{DATA_PATH}/train'
MDLS_PATH = f'./models_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)

def seed_all(seed=0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state    

random_state = seed_all(PARAMS['seed'])

start_time = time.time()

In [ ]:
if DEBUG:
    df = pd.read_csv(f'{DATA_PATH}/train_labels.csv').sample(100).reset_index(drop=True)
else:
    df = pd.read_csv(f'{DATA_PATH}/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'{IMGS_PATH}/{x[0]}/{x}.npy')
df.head()

In [ ]:
train_aug = A.Compose([
    A.Resize(PARAMS['img_size'], 
             PARAMS['img_size'], 
             cv2.INTER_NEAREST),
    A.VerticalFlip(p=.5),
    A.HorizontalFlip(p=.5),
    A.Transpose(p=.25),
    A.ShiftScaleRotate(p=.25, rotate_limit=0)
])
val_aug = A.Compose([
    A.Resize(PARAMS['img_size'], 
             PARAMS['img_size'], 
             cv2.INTER_NEAREST),
])

In [ ]:
#!g1.1
class ClassificationDataset:
    
    def __init__(self, img_paths, targets, aug, tta=None): 
        self.img_paths = img_paths
        self.targets = targets
        self.aug = aug
        self.tta = tta

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):      
        img = np.load(self.img_paths[item]).astype(np.float)
        targets = self.targets[item]
        img = np.vstack(img).astype(np.float)
        img = self.aug(image=img)['image'][np.newaxis, ]
        if self.tta: 
            img = flip(img, axis=self.tta)
        return {
            'image': torch.tensor(img.copy(), dtype=torch.float),
            'targets': torch.tensor(targets, dtype=torch.float),
        }

In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
#!g1.1
class EffNet(nn.Module):
    
    def __init__(self, params, out_dim, infer=False):
        super(EffNet, self).__init__()
        if infer:
            self.enet = enet.EfficientNet.from_name(params['backbone'])
        else:
            self.enet = enet.EfficientNet.from_pretrained(params['backbone'])
        nc = self.enet._fc.in_features
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(
            in_channels=1 if params['mixup'] else 6, 
            out_channels=3, 
            kernel_size=3, 
            stride=1, 
            padding=3, 
            bias=False
        )
        if params['dropout']:
            self.myfc = nn.Sequential(
                nn.Dropout(params['dropout']),
                nn.Linear(nc, int(nc / 4)),
                nn.Dropout(params['dropout']),
                nn.Linear(int(nc / 4), out_dim)
            )
        else:
            self.myfc = nn.Linear(nc, out_dim)
        
    
    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [ ]:
#!g1.1
def train(data_loader, model, optimizer, device):
    model.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='training'):
        inputs = data['image']
        targets = data['targets']
        if PARAMS['mixup']:
            inputs, targets_a, targets_b, lam = mixup_data(
                inputs, 
                targets.view(-1, 1), 
                use_cuda=True
            )
            inputs = inputs.to(device, dtype=torch.float)
            targets_a = targets_a.to(device, dtype=torch.float)
            targets_b = targets_b.to(device, dtype=torch.float)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = mixup_criterion(
                nn.BCEWithLogitsLoss(), 
                outputs, targets_a, targets_b, lam
            )
        else:
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    val_targets = []
    val_outputs = []
    with torch.no_grad():
        for data in tqdm(data_loader, position=0, leave=True, desc='evaluating'):
            inputs = data['image']
            targets = data['targets']
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            output = model(inputs).sigmoid()
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            val_targets.extend(targets)
            val_outputs.extend(output)
    return val_outputs, val_targets

In [ ]:
skf = StratifiedKFold(PARAMS['folds'], shuffle=True, random_state=PARAMS['seed'])
df['fold'] = -1
for i, (train_idxs, val_idxs) in enumerate(skf.split(df, df['target'])):
    df.loc[val_idxs, 'fold'] = i
display(df.head())

In [ ]:
#!g1.1
preds_val = []

if DEBUG:
    n_folds_train = 2
else:
    n_folds_train = PARAMS['folds'] if not PARAMS['folds_train'] else PARAMS['folds_train']
start_folds_train = 0

for fold_num in range(start_folds_train, n_folds_train):
    print('=' * 20, 'FOLD:', fold_num, '=' * 20)
    train_idxs = np.where((df['fold'] != fold_num))[0]
    val_idxs = np.where((df['fold'] == fold_num))[0]
    train_imgs, val_imgs = df.loc[train_idxs, 'img_path'].values, df.loc[val_idxs, 'img_path'].values
    train_targets, val_targets = df.loc[train_idxs, 'target'].values, df.loc[val_idxs, 'target'].values
    train_dataset = ClassificationDataset(
        img_paths=train_imgs, 
        targets=train_targets,
        aug=train_aug
    )
    val_dataset = ClassificationDataset(
        img_paths=val_imgs, 
        targets=val_targets,
        aug=val_aug
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=PARAMS['batch_size'],
        shuffle=True, 
        num_workers=PARAMS['workers']
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, 
        batch_size=PARAMS['batch_size'],
        shuffle=False, 
        num_workers=PARAMS['workers']
    )
    model = EffNet(params=PARAMS, out_dim=1)
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=PARAMS['lr'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, PARAMS['epochs'])
    print('train len:', len(train_dataset),'| val len:', len(val_dataset))
    best_file = '{}/model_best_{}.pth'.format(MDLS_PATH, fold_num)
    roc_auc_max = 0
    epochs_no_improve = 0
    for epoch in tqdm(range(PARAMS['epochs']), desc='epochs'):
        print(time.ctime(), 'epoch:', epoch)
        train(train_loader, model, optimizer, device=DEVICE)
        preds, val_targets = evaluate(val_loader, model, device=DEVICE)
        roc_auc = metrics.roc_auc_score(val_targets, preds)
        scheduler.step()
        content = '{} epoch {}, lr: {:.8f}, ROC-AUC: {:.4f}'.format(
            time.ctime(),
            epoch, 
            optimizer.param_groups[0]['lr'], 
            roc_auc
        )
        print(content)
        with open('{}/log_{}.txt'.format(MDLS_PATH, fold_num), 'a') as appender:
            appender.write(content + '\n')
        if roc_auc > roc_auc_max:
            torch.save(model.state_dict(), best_file)
            print('ROC-AUC improved {:.4f} --> {:.4f} model saved'.format(roc_auc_max, roc_auc))
            roc_auc_max = roc_auc
            preds_best = np.squeeze(preds)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PARAMS['patience']:
            print('no improve for', epochs_no_improve, 'epochs | early stopping')
            early_stop = True
            break
    preds_val.extend(preds_best)
    torch.save(
        model.state_dict(), 
        os.path.join('{}/model_final_{}.pth'.format(MDLS_PATH, fold_num))
    )
    del model, train_dataset, val_dataset, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
roc_auc_max, th_max = 0, 0
target_val = df.target.values
for th in np.linspace(.1, 1, 100):
    try:
        roc_auc = metrics.roc_auc_score(np.array(preds_val) > th, target_val)
        if roc_auc > roc_auc_max:
            roc_auc_max = roc_auc
            th_max = th
    except:
        pass
print('ROC-AUC max:', np.round(roc_auc_max, 2), '| th max:', np.round(th_max, 2))
with open(f'{MDLS_PATH}/th.json', 'w') as file:
    json.dump({
        'ROC-AUC max': roc_auc_max, 
        'th max': th_max,
    }, file)